In [1]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
pip install sentinelhub

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [3]:
import ee
import folium
ee.Authenticate()
ee.Initialize()

In [4]:
# Custom function to add Earth Engine layers to folium maps
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr="Google Earth Engine",
        name=name,
        overlay=True,
        control=True,
    ).add_to(self)

# Add the custom method to folium
folium.Map.add_ee_layer = add_ee_layer

In [5]:
# Define the bounding box for Sri Lanka(Area of Interest)
min_lon = 79.6  # Minimum longitude (west)
min_lat = 5.9   # Minimum latitude (south)
max_lon = 81.9  # Maximum longitude (east)
max_lat = 9.8   # Maximum latitude (north

In [6]:
# Define the region of interest (Sri Lanka)
min_lon, min_lat, max_lon, max_lat = 79.5, 5.8, 82.0, 9.8
region = ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat])

In [7]:
# Sentinel-1 data (gridded SAR data)
sentinel1 = ee.ImageCollection("COPERNICUS/S1_GRD")

# Filter by date and region
filtered_sentinel1 = sentinel1.filterBounds(region).filterDate('2023-01-01', '2023-12-31')

# Calculate mean for VV and VH bands
vv_band = filtered_sentinel1.select('VV').mean()
vh_band = filtered_sentinel1.select('VH').mean()

In [8]:
# Visualization parameters for VV and VH bands
vv_vis = {
    'min': -26.78,
    'max': 31.88,
    'palette': ['blue', 'green', 'yellow', 'red']
}
vh_vis = {
    'min': -38.79,
    'max': 20.30,
    'palette': ['blue', 'green', 'yellow', 'red']
}

In [9]:
print(vv_band.getInfo())  # Prints metadata, including date range

{'type': 'Image', 'bands': [{'id': 'VV', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [10]:
print(vh_band.getInfo())  # Same for VH

{'type': 'Image', 'bands': [{'id': 'VH', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [11]:
print("Sentinel Data: ", filtered_sentinel1.size().getInfo())

Sentinel Data:  283


In [12]:
# Function to create monthly composites and visualize
def generate_monthly_maps():
    months = [f"{i:02}" for i in range(1, 13)]  # Generate '01' to '12'
    for month in months:
        # Filter data for the specific month
        start_date = f"2023-{month}-01"
        end_date = f"2023-{month}-28"  # Assuming February has up to 28 days
        monthly_data = filtered_sentinel1.filterDate(start_date, end_date)

        # Calculate monthly mean for VV and VH
        vv_monthly = monthly_data.select('VV').mean()
        vh_monthly = monthly_data.select('VH').mean()

        # Create maps
        vv_map = folium.Map(location=[min_lat, min_lon], zoom_start=6)
        vh_map = folium.Map(location=[min_lat, min_lon], zoom_start=6)

        vv_map.add_ee_layer(vv_monthly, vv_vis, f"VV Band - {start_date}")
        vh_map.add_ee_layer(vh_monthly, vh_vis, f"VH Band - {start_date}")

        folium.LayerControl().add_to(vv_map)
        folium.LayerControl().add_to(vh_map)

        # Display the maps
        print(f"VV Band Map for {month} 2023:")
        display(vv_map)

        print(f"VH Band Map for {month} 2023:")
        display(vh_map)

# Call the function to generate monthly maps
generate_monthly_maps()


VV Band Map for 01 2023:


VH Band Map for 01 2023:


VV Band Map for 02 2023:


VH Band Map for 02 2023:


VV Band Map for 03 2023:


...

VH Band Map for 10 2023:


VV Band Map for 11 2023:


VH Band Map for 11 2023:


VV Band Map for 12 2023:


VH Band Map for 12 2023:


In [30]:
import pandas as pd

# Sentinel-1 data (gridded SAR data)
sentinel1 = ee.ImageCollection("COPERNICUS/S1_GRD")

# Filter by region and date
filtered_sentinel1 = sentinel1.filterBounds(region).filterDate('2023-01-01', '2023-12-31')

# Calculate daily means
daily_means = filtered_sentinel1.map(
    lambda img: img.set('date', img.date().format('YYYY-MM-dd'))
).reduceColumns(
    ee.Reducer.toList(), ['date']
)


In [32]:
# Generate full date range for 2023
full_dates = pd.date_range(start='2023-01-01', end='2023-12-31')

# Get the exported Sentinel dates
exported_dates = pd.to_datetime(daily_means.getInfo()['list'])

# Identify missing dates
missing_dates = full_dates.difference(exported_dates)


In [34]:
task = ee.batch.Export.table.toDrive(
    collection=filtered_sentinel1,
    description='Sentinel1_Export',
    fileFormat='CSV'
)
task.start()
